<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Text_Generation_using_Fnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install datasets
!pip install torch[transformers]

In [2]:
import torch

In [3]:
from datasets import load_dataset
datasets = load_dataset('wikitext','wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [5]:
import re
def preprocess_text(sentence):
  text = sentence['text'].lower() # lowering the sentence and storing in text vaiable
  text = re.sub('[^a-z?!.,]',' ',text) # removing other than characters and punctuations
  text = re.sub('\s\s+',' ',text) # removing double spaces
  sentence['text'] = text
  return sentence

In [6]:
datasets['train'] = datasets['train'].map(preprocess_text)
datasets['test'] = datasets['test'].map(preprocess_text)
datasets['validation'] = datasets['validation'].map(preprocess_text)

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [7]:
datasets['train'] = datasets['train'].filter(lambda x : len(x['text']) > 20)
datasets['test'] = datasets['test'].filter(lambda x : len(x['text']) > 20)
datasets['validation'] = datasets['validation'].filter(lambda x : len(x['text']) > 20)

Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4358 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 2312
    })
    train: Dataset({
        features: ['text'],
        num_rows: 18794
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1988
    })
})

In [41]:
 from torchtext.data.utils import get_tokenizer
 tokenizer = get_tokenizer("spacy",unk_token='<unk>')
 tokenizer('I have two asfdf')

TypeError: ignored

In [10]:

from collections import Counter,OrderedDict


counter = Counter()
for lines in datasets['train']:
    counter.update(tokenizer(lines['text']))


In [19]:

from torchtext.vocab import vocab
from torchtext.vocab import Vectors

special_tokens = ['<pad>', '<bos>', '<eos>', '<unk>']
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab_obj = vocab(ordered_dict, specials = special_tokens)

In [22]:
vec = torchtext.vocab.GloVe(name='6B', dim=50)

.vector_cache/glove.6B.zip: 862MB [02:38, 5.43MB/s]                           
100%|█████████▉| 399999/400000 [00:12<00:00, 32351.97it/s]


In [25]:
examples = ['chip', 'baby', 'Beautiful','<pad>', '<bos>', '<eos>', '<unk>']
ret = vec.get_vecs_by_tokens(examples, lower_case_backup=True)


In [26]:
ret

tensor([[-0.7710, -1.1697,  1.5195,  0.8371,  0.7419, -0.2185, -0.7212, -0.9400,
         -0.0113,  0.5485,  0.4040, -0.1846, -0.4630,  0.2620, -0.6464,  0.3599,
         -0.8610, -0.3869, -0.0271, -1.0254,  0.3280, -0.7500, -0.6859, -0.6912,
          0.3429, -0.6660, -0.2910, -0.6104,  0.3322, -0.4252,  2.4573, -0.8748,
          0.4891,  1.2888,  0.5780, -0.5509, -0.2263,  0.8127,  0.7048, -0.5498,
          0.3620, -0.2171, -0.2991,  0.2917,  1.2260,  0.2446,  1.2133, -0.0967,
          0.0474,  0.1971],
        [ 0.5494,  0.2299, -0.0357, -0.9143,  0.7044,  1.3736, -0.9937, -0.5034,
          0.5793,  0.3481,  0.2385,  0.5444,  0.3432,  0.5741,  1.3732,  0.4636,
         -0.7288,  0.2887,  0.1001, -0.2302, -0.1289,  0.7033,  0.3961,  0.2605,
          0.2697, -1.3036, -0.9377,  0.2705,  0.6070, -0.6689,  1.9709,  0.6796,
         -0.6944,  1.0380,  0.5136,  0.2302,  0.3646, -0.3090,  1.1395, -1.1466,
         -0.7889,  0.0544, -0.0691, -0.2439,  1.4049,  0.0919,  0.2313, -1.3028,


In [40]:
vocab_obj(tokenizer('asfbc hi '))

RuntimeError: ignored

In [47]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))



In [48]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

In [50]:
wrapped_tokenizer('hi this is')

Exception: ignored

In [52]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [55]:
tokenizer('hi this is abcsdfsaf ')

{'input_ids': [5303, 428, 318, 450, 6359, 7568, 49585], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [57]:
len(tokenizer.vocab)

50257

In [59]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [60]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [61]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [69]:
tokenizer('hi this is HOOOGafsadfjdkslfjdsfjdsklfjdskljfsdaklfjdsklf4324IING ')

{'input_ids': [101, 7632, 2023, 2003, 7570, 9541, 3654, 10343, 4215, 2546, 3501, 2094, 5705, 10270, 3501, 5104, 2546, 3501, 5104, 2243, 10270, 3501, 5104, 2243, 2140, 3501, 10343, 23597, 10270, 3501, 5104, 2243, 10270, 23777, 18827, 6137, 3070, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [71]:
tokenizer.convert_ids_to_tokens(tokenizer('hi this is HOOOGafsadfjdkslfjdsfjdsklfjdskljfsdaklfjdsklf4324IING ')['input_ids'])

['[CLS]',
 'hi',
 'this',
 'is',
 'ho',
 '##oo',
 '##ga',
 '##fs',
 '##ad',
 '##f',
 '##j',
 '##d',
 '##ks',
 '##lf',
 '##j',
 '##ds',
 '##f',
 '##j',
 '##ds',
 '##k',
 '##lf',
 '##j',
 '##ds',
 '##k',
 '##l',
 '##j',
 '##fs',
 '##dak',
 '##lf',
 '##j',
 '##ds',
 '##k',
 '##lf',
 '##43',
 '##24',
 '##ii',
 '##ng',
 '[SEP]']

In [75]:
len(tokenizer.vocab)

30522